In [1]:
import operator
import random
from IPython.display import clear_output

In [2]:
# путь к файлу с исходными данными
path = "9.txt"

data = []

# грузоподъемность рюкзака
maxW = 0
# ценность рюкзака
maxV = 0


fRow = True
with open(path, "r") as file:
    for i in file:
        # первая строка с параметрами рюкзака
        if (fRow):
            fRow = False
            maxW, maxV = i.split()
            # кастуем к integer
            maxW = int(maxW)
            maxV = int(maxV)
        # остальные строки с параметрами предметов
        else:
            data.append(i.split())

print("Грузоподъемность рюкзака: " + str(maxW))
print("Вместимость рюкзака: " + str(maxV))

Грузоподъемность рюкзака: 13000
Вместимость рюкзака: 12


In [3]:
# вес
w = []
# объем
v = []
# цэ - цЭнность :)
c = []

for i in range(len(data)):
    w.append(int(data[i][0]))
    v.append(float(data[i][1]))
    c.append(int(data[i][2]))
    
data = list(zip(w, v, c))

In [4]:
def randomParents(start, stop, count):
    # Получаем неповторяющиеся случайные числа
    gamma = []
    for i in range(0, count):
        while True:
            item = random.randint(start, stop)
            if not gamma.count(item):
                gamma.append(item)
                yield item
                break

In [5]:
def paramsFromGenome(genome):
    w = 0
    v = 0
    c = 0
    for i in range(len(genome)):
        if(int(genome[i])):
            w += data[i][0]
            v += data[i][1]
            c += data[i][2]
    return {"genome": genome,
            "w": w,
            "v": v,
            "c": c}

In [6]:
def generateIndivid(itemArr):
    individ = ""
    w = 0
    v = 0
    c = 0
    
    for i in range(len(itemArr)):
        rnd = random.randint(0, 1)
        if (rnd):
            w += itemArr[i][0]
            v += itemArr[i][1]
            c += itemArr[i][2]
            
        individ += str(rnd)
    return {"genome": individ,
            "w": w,
            "v": v,
            "c": c}

In [7]:
def selection(population):
    totalCost = sum(i["c"] for i in population)
    populationCrossover = []
    for index, individual in enumerate(population):
        probability = individual["c"] / totalCost
        roulette = random.random()
        if (probability <= roulette and individual["v"] <= maxV and individual["w"] <= maxW):
            populationCrossover.append(individual)
    return populationCrossover

In [8]:
def merge(p1, p2, rDots):
    
    fChildGenome = p1["genome"][:rDots[0]] + p2["genome"][rDots[0]:rDots[1]] + p1["genome"][rDots[1]:rDots[2]] + p2["genome"][rDots[2]:]
    sСhildGenome = p2["genome"][:rDots[0]] + p1["genome"][rDots[0]:rDots[1]] + p2["genome"][rDots[1]:rDots[2]] + p1["genome"][rDots[2]:]
    
    fChild = paramsFromGenome(fChildGenome)
    sChild = paramsFromGenome(sСhildGenome)
    
    return [fChild, sChild]
    
def reproduction(parentPopulation):
    randomQueue = list(randomParents(0, len(parentPopulation) - 1, len(parentPopulation)))
    randomDots = sorted(list(randomParents(0, 28, 3)))
    newPopulation = []
    alone = 0
    for i in range(0, len(randomQueue), 2):
        try:
            newPopulation = newPopulation + merge(parentPopulation[randomQueue[i]], parentPopulation[randomQueue[i + 1]], randomDots)
        except:
            alone += 1
    return newPopulation

In [9]:
def mutation(population, pp=10):
    resPop = population
    individs = list(randomParents(0, len(population) - 1, int(len(population) / 10)))
    for i in individs:
        ones = []
        for ch in range(len(resPop[i]["genome"])):
            if (resPop[i]["genome"][ch] == "0"):
                ones.append(ch)
        mutated = ones[random.randint(0, len(ones) - 1)]
        resPop[i]["genome"] = resPop[i]["genome"][:(mutated)] + "1" + resPop[i]["genome"][(mutated + 1):]

    return resPop

In [10]:
def newPopGen(parent, mutated):
    nParent = []
    for i in parent:
        i["c"] = i["c"] * 0.8
        nParent.append(i)
    newPopulation = nParent + mutated
    newPopulation = sorted(newPopulation, key = lambda i: i['c'],reverse=True)
    newPopulation = newPopulation[:200]
    
    index = 0
    while (len(newPopulation) < 200):
        nInd = newPopulation[index]
        newPopulation.append(nInd)
        index += 1
    return newPopulation
    

In [11]:
generateIndivid(data)
populationNumber = 200
population = []

for i in range(populationNumber):
    population.append(generateIndivid(data))

In [12]:
iterCount = 500

for i in range(iterCount):
    print("Итерация: " + str(i) + "/" + str(iterCount))
    
    parentPopulation = selection(population)
    childPopulation = reproduction(parentPopulation)
    mutatedPopulation = mutation(childPopulation)
    population = newPopGen(parentPopulation, mutatedPopulation)
    
    clear_output()

population = selection(population)
population = sorted(population, key = lambda i: i["c"],reverse=True)
print(population[0]["c"])
print(population[0]["v"])
print(population[0]["w"])

4431
10.8
12985
